# Pick and place loop with branching

This example notebook demonstrates two pick-and-place processes:

- Moving one or more objects in a loop.
- Moving a single object back and forth using a loop and branching.

In particular, this example covers:

- Behavior tree nodes
    - `Loop` with `SubTreeCondition`
    - `Loop` with max iteration count
    - `Branch` with a `SubTreeCondition` and `then` and `else` children

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [portal.intrinsic.ai](https://portal.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `building_block:building_block_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `building_block:building_block_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect to your solution and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world
simulator = solution.simulator
pose_estimators = solution.pose_estimators

Next, create the required skill instances:

In [ ]:
collisions_disabled = skills.move_robot.CollisionSettings(disable_collision_checking=True)

enable_gripper = skills.enable_gripper(clear_faults=True)

move_home = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=world.robot.joint_configurations.home)])

move_view_pose_left = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_left)])

move_view_pose_right = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_right)])

# Moves the building block in the belief world to its estimated position from
# camera image of the simulated/real world.
estimate_pose = skills.estimate_and_update_pose(
        pose_estimator=pose_estimators.building_block_ml_estimator,
        object=world.building_block0,
        camera=resources.wrist_camera)

# Moves the global frames `world.grasp_frame` and `world.pregrasp_frame` to
# suitable grasp/pregrasp positions above the building block.
plan_grasp = skills.plan_grasp(
    objects=[world.building_block0],
    product_part_name='building_block',
    tool_frame=world.picobot_gripper.tool_frame,
    advanced_params=skills.plan_grasp.PlanGraspAdvancedParams(
        output_pregrasp_frame=world.pregrasp_frame,
        output_grasp_frame=world.grasp_frame,
      )
    )

move_pregrasp = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=skills.move_robot.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.pregrasp_frame))])

# Move to grasp pose: Disable collision checking because at the end of the move
# the gripper tip will touch the building block.
move_grasp_unsafe = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=skills.move_robot.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.grasp_frame),
            motion_type=skills.move_robot.MotionSegment.LINEAR,
            collision_settings=collisions_disabled)])

# Turn on the suction gripper which will "attach" the building block in the
# simulation/real world.
grasp = skills.control_suction_gripper(
    grasp=skills.control_suction_gripper.GraspRequest())

# Reparent building block to the robot in the belief world so that the building block
# in the belief world will move together with the robot.
attach_block = skills.attach_object_to_robot(
    gripper_entity=world.picobot_gripper,
    object_entity=world.building_block0)

# Retract up by 3cm. Disable collision checking because we expect collisions at the beginning
# of the move.
# - Right after grasping, the attached block is in collision with the tray.
# - Right after releasing, the gripper tip is in collision with the building block.
move_up_unsafe = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=skills.move_robot.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.picobot_gripper.tool_frame,
                target_frame_offset=data_types.Pose3(translation=[0,0,-0.03])),
            motion_type=skills.move_robot.MotionSegment.LINEAR,
            collision_settings=collisions_disabled)])

drop_offset = data_types.Pose3(
    rotation=data_types.Rotation3.from_euler_angles(rpy_degrees=[180, 0, 90]),
    translation=[0,0,0.05])

move_drop_left = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=skills.move_robot.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.target_left,
                target_frame_offset=drop_offset))])

move_drop_right = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=skills.move_robot.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.target_right,
                target_frame_offset=drop_offset))])

# Turn off the suction gripper which will "drop" the building block in the
# simulation/real world.
release = skills.control_suction_gripper(
    release=skills.control_suction_gripper.ReleaseRequest())

# Reparent building block back to the world root in the belief world.
detach = skills.detach_object(
    gripper_entity=world.picobot_gripper,
    object_entity=world.building_block0)

## Pick and place all objects that can be detected

Create and run a behavior tree that picks and places the object detected by `estimate_pose` in a loop. The loop exits once `estimate_pose` fails (=no more object can be detected):

In [ ]:
world.reset()

initialize = bt.SubTree(
    name='Initialize',
    behavior_tree=bt.Sequence([
        bt.Task(action=enable_gripper, name='Enable gripper'),
        bt.Task(action=move_home, name='Move home')]))

find_object = bt.SubTree(
    name='Find object',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_view_pose_left, name='Move to view pose left'),
        bt.Task(action=estimate_pose, name='Estimate pose')]))

pick_object = bt.SubTree(
    name='Pick object',
    behavior_tree=bt.Sequence([
        bt.Task(action=plan_grasp, name='Plan grasp'),
        bt.Task(action=move_pregrasp, name='Move to pregrasp'),
        bt.Task(action=move_grasp_unsafe, name='Move to grasp'),
        bt.Task(action=grasp, name='Grasp block'),
        bt.Task(action=attach_block, name='Attach block'),
        bt.Task(action=move_up_unsafe, name='Move up after grasp')]))

place_object = bt.SubTree(
    name='Place object',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_drop_right, name='Move to drop'),
        bt.Task(action=release, name='Release block'),
        bt.Task(action=detach, name='Detach block'),
        bt.Task(action=move_up_unsafe, name='Move up after drop')]))

loop = bt.Loop(
    while_condition=bt.SubTreeCondition(find_object),
    do_child=bt.Sequence([pick_object, place_object]))

pick_and_place_loop = bt.BehaviorTree(
    name='Pick and place in loop',
    root=bt.Sequence([
        initialize,
        loop,
        bt.Task(action=move_home, name='Move back home')]))

executive.run(pick_and_place_loop)

## Pick and place back and forth

Create and run a behavior tree that moves an object back and forth by repeatedly doing:

1. Move to the left spot. If there is an object (=`estimate_pose` succeeds), move it to the right spot.
2. Move to the right spot. If there is an object (=`estimate_pose` succeeds), move it to the left spot.

In [ ]:
world.reset()
simulator.reset()

initialize = bt.SubTree(
    name='Initialize',
    behavior_tree=bt.Sequence([
        bt.Task(action=enable_gripper, name='Enable gripper'),
        bt.Task(action=move_home, name='Move home')]))

find_object_left = bt.SubTree(
    name='Find object left',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_view_pose_left, name='Move to view pose left'),
        bt.Task(action=estimate_pose, name='Estimate pose')]))

find_object_right = bt.SubTree(
    name='Find object right',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_view_pose_right, name='Move to view pose right'),
        bt.Task(action=estimate_pose, name='Estimate pose')]))

pick_object = bt.SubTree(
    name='Pick object',
    behavior_tree=bt.Sequence([
        bt.Task(action=plan_grasp, name='Plan grasp'),
        bt.Task(action=move_pregrasp, name='Move to pregrasp'),
        bt.Task(action=move_grasp_unsafe, name='Move to grasp'),
        bt.Task(action=grasp, name='Grasp block'),
        bt.Task(action=attach_block, name='Attach block'),
        bt.Task(action=move_up_unsafe, name='Move up after grasp')]))

place_object_left = bt.SubTree(
    name='Place object left',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_drop_left, name='Move to drop left'),
        bt.Task(action=release, name='Release block'),
        bt.Task(action=detach, name='Detach block'),
        bt.Task(action=move_up_unsafe, name='Move up after drop')]))

place_object_right = bt.SubTree(
    name='Place object right',
    behavior_tree=bt.Sequence([
        bt.Task(action=move_drop_right, name='Move to drop right'),
        bt.Task(action=release, name='Release block'),
        bt.Task(action=detach, name='Detach block'),
        bt.Task(action=move_up_unsafe, name='Move up after drop')]))

move_back_or_forth_once = bt.Branch(
    name='Move block right else move block left',
    if_condition=bt.SubTreeCondition(find_object_left),
    then_child=bt.Sequence([pick_object, place_object_right]),
    else_child=bt.Branch(
        if_condition=bt.SubTreeCondition(find_object_right),
        then_child=bt.Sequence([pick_object, place_object_left]),
        else_child=bt.Fail(failure_message='No object found')))

move_back_and_forth = bt.Loop(max_times=3, do_child=move_back_or_forth_once)

back_and_forth_branched = bt.BehaviorTree(
    name='Back and forth branched',
    root=bt.Sequence([
        initialize,
        move_back_and_forth,
        bt.Task(action=move_home, name='Move back home')]))

executive.run(back_and_forth_branched)